In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

import warnings
warnings.filterwarnings('error')

import sys
sys.path.append('../../')
from feature import selection as sel

from sklearn.feature_selection import VarianceThreshold

FEATURES_PATH =  '../../datasets/features_data/'

Feature selection = Pearson correlation
- Calculate correlation between fault (then severity) and features by axis
- Transform faults to binary vector (is unbalance, is not unbalance) for each fault - one hot encoding

Time domain

In [ ]:
def show_time_domain_correlation(corr_table):
    num_of_faults = len(corr_table['fault'].cat.categories)
    fig, axes = plt.subplots(2, num_of_faults // 2, figsize=(20, 10))

    for i, group in enumerate(corr_table.groupby(by='fault', observed=True)):
        fault, rows = group
        x = rows.sort_values(by='corr', ascending=False)
        sel.plot_bar_chart(axes.flatten()[i], x['feature'], x['corr'], f'Fault: {fault}')

    for ax in axes[-1]:
        ax.set_xlabel('Feature')
    for ax in axes[:,0]:
        ax.set_ylabel('Correlation with fault')

    fig.tight_layout()

Variance Threshold in Time domain

In [ ]:
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]


X_train = sel.load_td_feat(['az'], all=True, path=FEATURES_PATH)
variance_threshold_selector(sel.filter_out_metadata_columns(X_train), 3)

Ordered importance of features to fault based on their Pearson correlation coeficient

In [ ]:
features = sel.load_td_feat(['az'], all=True, path=FEATURES_PATH)
columns = sel.filter_out_metadata_columns(features).columns
df = sel.corr_features_to_fault(features, columns)
show_time_domain_correlation(df)

Correlations among faults and time domain featured unordered

In [ ]:
corr_to_class = df.pivot(index='fault', columns='feature', values='corr')
fig, ax = plt.subplots(figsize=(10, 5))
sb.heatmap(corr_to_class, annot=True, cmap='Greens', ax=ax)
plt.show()

Rank order of features averaged among all fault types
- *Less is better*

In [ ]:
corr_fault_to_feat = df.pivot(index='fault', columns='feature', values='corr')

feature_ranks = corr_fault_to_feat.rank(axis='columns', method='dense', ascending=False)
common_rank = feature_ranks.mean().sort_values().to_frame(name='rank')
common_rank.plot.bar(grid=True, legend=False)
plt.show()

Frequency domain

In [ ]:
def show_freq_domain_correlation(corr_table):
    num_of_faults = len(corr_table['fault'].cat.categories)
    num_of_windows = len(corr_table['window'].cat.categories)
    
    fig, axes = plt.subplots(num_of_faults, num_of_windows, figsize=(25, 40))

    for i, group in enumerate(corr_table.groupby(by=['fault', 'window'], observed=True)):
        grouper, rows = group
        fault, win = grouper
        x = rows.sort_values(by='corr', ascending=False)
        sel.plot_bar_chart(axes.flatten()[i], x['feature'], x['corr'], f'Fault: {fault} \n(Window = {win})')
    
    for ax in axes[-1]:
        ax.set_xlabel('Feature')
    for ax in axes[:,0]:
        ax.set_ylabel('Correlation with fault')

    fig.tight_layout()

How does importance of features change with window size
- -> Increasing window size
- v Different faults 

In [ ]:
features = sel.load_fd_feat(['az'], all=True, path=FEATURES_PATH)
features = sel.calc_corr_in_fft_windows(features, sel.FD_COLUMNS)
show_freq_domain_correlation(features)

Most important features by fault type (rank averaged by window size)
- *Less is better*

In [ ]:
ranks = sel.weighted_rank_features_corr(features, index='window', weighted=False)
sel.plot_ranked_features(ranks)

Rank order of features averaged among all fault types
- *Less is better*

In [ ]:
corr_fault_to_feat = ranks.reset_index().pivot(index='fault', columns='feature', values='rank')
common_rank = feature_ranks.mean().sort_values().to_frame(name='rank')
common_rank.plot.bar(grid=True, legend=False)
plt.show()
corr_fault_to_feat

Wavelets

In [ ]:
def show_wpd_domain_correlation(corr_table, top_n):
    num_of_faults = len(corr_table['fault'].cat.categories)
    num_of_windows = len(corr_table['metric'].cat.categories)
    
    fig, axes = plt.subplots(num_of_faults, num_of_windows, figsize=(25, 40))

    for i, group in enumerate(corr_table.groupby(by=['fault', 'metric'], observed=True)):
        grouper, rows = group
        fault, metric = grouper
        x = rows.sort_values(by='corr', ascending=False).head(top_n)
        sel.plot_bar_chart(axes.flatten()[i], x['feature'], x['corr'], f'Fault: {fault} \n({metric})')

    for ax in axes[-1]:
        ax.set_xlabel('Feature')
    for ax in axes[:,0]:
        ax.set_ylabel('Correlation with fault')

    fig.tight_layout()

WPD all layers (no simple conclusions can be drawn)

In [ ]:
features = sel.load_wavelet_domain_features(['ax', 'ay', 'az'])
features = sel.calc_corr_in_wpd_features(features)
features

In [ ]:
show_wpd_domain_correlation(features, top_n=25)

WPD correlations in one chosen layer

In [ ]:
level = 4
layer = features[features['feature'].str.startswith(f'L{level}')]
show_wpd_domain_correlation(layer, top_n=25)

Most important features by fault type (weigthed rank averaged by metrics)
- *Less is better*

In [ ]:
ranks = sel.weighted_rank_features_corr(layer, index='metric', weighted=True)
sel.plot_ranked_features(ranks)

Different WPD layer

In [ ]:
level = 3
layer = features[features['feature'].str.startswith(f'L{level}')]
ranks = sel.weighted_rank_features_corr(layer, index='metric', weighted=True)
sel.plot_ranked_features(ranks)